In [ ]:
# load libraries and scripts
import JSON
using Plots
using HDF5
using Trapz
include("../scripts/tools.jl")
include("../scripts/get_grid.jl")
include("../scripts/get_params.jl")
include("../scripts/disturbances.jl")
include("../scripts/stable.jl")
include("../scripts/dynamics.jl")

In [ ]:
#load borders
border = import_border("../data/borders/border.json")

# create the lattice grid
dx = 20
Nx, Ny, xrange, yrange, isinside, isborder, n = get_grid(border, Float64(dx))

# define x and y coordinate for plotting
x = repeat(reshape(xrange,1,Nx), Ny, 1)
y = repeat(reshape(yrange,Ny,1), 1, Nx);

In [ ]:
# compute or load grid parameters
recompute_param = true
if(recompute_param || isfile("../numerics/grid_params_" * string(dx) * ".h5"))
    bx, by, p, m, d = get_params_diff(isinside, n, Float64(dx), yrange, xrange, "../data/disc/pantagruel.h5",
        "../numerics/grid_params_" * string(dx) * ".h5", Niter = 10000, dmax = 10.0, patch=0.003, bmin=0.01)
    #bx, by, p, m, d = get_params(isinside, isborder, Float64(dx), yrange, xrange, "../data/pantagruel.h5",
    #    "../numerics/grid_params_" * string(dx) * ".h5")
else
    bx, by, p, m, d = get_params(isinside, "../numerics/grid_params_" * string(dx) * ".h5")
end

In [ ]:
# compute, iteratively, the stable solution
th0, ths = compute_stable_sol(isinside, n, bx, by, p, Niter = 20000, interval = 1000);

In [ ]:
# define a disturbance
# dP = -9.0
dP = 0.0
sigma = 50.0
location = [-1500., -900.]
dp = local_disturbance(isinside, xrange, yrange, location, dP, sigma)
println(trapz((yrange, xrange), dp))
ctr_plot(isinside, dp)

In [ ]:
# perform a dynamical simulation
ts, thetas, omegas = perform_dyn_sim(isinside, n, bx, by, p+dp, m, d, th0, interval = 1000, Ndt = Int64(2E5), dt = 0.0001);

In [ ]:
# plotting the frequency at different locations
coord = [0 -250; 0.0 0.0; -500 500; 500. 500.; -1500.0 -1000.0; 750.0 -1000]
time_plot(ts, omegas, coord, tstart = 0.0, tend = 5.0, borders = [border])

In [ ]:
# evolution of the frequency
ctr_movie(ts, omegas, tstart = 0.0, tend = 50.0)

In [ ]:
sum(p[isinside .| isborder])